In [23]:
import os 

In [24]:
%pwd

'/home/ahmed/project'

In [25]:
os.chdir('../')

In [26]:
%pwd

'/home/ahmed'

In [27]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [28]:
from project.constants import *
from project.utils import create_directories,read_yaml
import time

In [29]:
class ConfigerationManager:
    def __init__(self, config=CONFIG_YAML_FILE, param=PARAM_YAML_FILE):
        self.config = read_yaml(config)
        self.param = read_yaml(param)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self):
        config = self.config.prepare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callback_config


In [30]:
import tensorflow as tf

In [31]:
class CallBacks:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)

    @property
    def create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [self.create_tb_callback, self.create_ckpt_callback]



In [33]:
# Test
try:
    config = ConfigerationManager()
    callbacks_config = config.get_prepare_callback_config()
    callback = CallBacks(callbacks_config)
    callback_list = callback.get_tb_ckpt_callbacks()
    # print(callback_list)

except Exception as e:
    raise e


In [ ]:
import os
import time
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf

from project.constants import *
from project.utils import create_directories, read_yaml


@dataclass(frozen=True)
class PrepareCallbackConfig:
    """
    Dataclass for storing the configuration required to create callbacks.

    Attributes:
        root_dir (Path): Base directory for callback-related artifacts.
        tensorboard_root_log_dir (Path): Directory where TensorBoard logs will be saved.
        checkpoint_model_filepath (Path): Full filepath where the model checkpoint will be stored.
    """
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


class ConfigerationManager:
    """
    Responsible for reading YAML configuration files and preparing
    callback-related directory paths.

    Args:
        config (str): Path to the configuration YAML file.
        param (str): Path to the params YAML file.

    Methods:
        get_prepare_callback_config: Returns a structured dataclass containing
                                     all callback-related paths.
    """

    def __init__(self, config=CONFIG_YAML_FILE, param=PARAM_YAML_FILE):
        """Load YAML config files and create root artifact directory."""
        self.config = read_yaml(config)
        self.param = read_yaml(param)

        # Create root artifacts directory
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        """
        Prepare and return callback configuration dataclass.

        Steps:
            - Read prepare_callbacks section from YAML.
            - Extract directory for model checkpoint.
            - Create TensorBoard log directory and checkpoint directory.
            - Wrap them inside PrepareCallbackConfig dataclass.

        Returns:
            PrepareCallbackConfig: Contains formatted directory/file paths.
        """
        config = self.config.prepare_callbacks

        # Extract folder path from full checkpoint filepath
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        # Ensure required directories exist
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        return PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )


class CallBacks:
    """
    Creates TensorBoard and ModelCheckpoint callbacks for training.

    Args:
        config (PrepareCallbackConfig): Dataclass containing callback paths.

    Methods:
        create_tb_callback (property): Returns TensorBoard callback.
        create_ckpt_callback (property): Returns ModelCheckpoint callback.
        get_tb_ckpt_callbacks: Returns list of both callbacks.
    """

    def __init__(self, config: PrepareCallbackConfig):
        """Store callback configuration."""
        self.config = config

    @property
    def create_tb_callback(self) -> tf.keras.callbacks.TensorBoard:
        """
        Create and return a TensorBoard callback.

        Behavior:
            - Creates a unique timestamped log directory inside the root log dir.
            - Helps visualize training metrics like loss, accuracy, graphs.

        Returns:
            TensorBoard: Keras TensorBoard callback instance.
        """
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)

    @property
    def create_ckpt_callback(self) -> tf.keras.callbacks.ModelCheckpoint:
        """
        Create and return a ModelCheckpoint callback.

        Behavior:
            - Saves only the best model based on validation performance.
            - Stores model file at the configured checkpoint path.

        Returns:
            ModelCheckpoint: Keras model checkpoint callback instance.
        """
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self) -> list:
        """
        Returns a list of callbacks required during training.

        Returns:
            list: Contains TensorBoard and ModelCheckpoint callbacks.
        """
        return [self.create_tb_callback, self.create_ckpt_callback]


# Testing
try:
    config = ConfigerationManager()
    callbacks_config = config.get_prepare_callback_config()
    callback = CallBacks(callbacks_config)
    callback_list = callback.get_tb_ckpt_callbacks()
    print(callback_list)
except Exception as e:
    raise e
